In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import os
import torch
import pandas as pd
import csv
import logging
import re
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
info = pd.read_csv('/home/duyongkang/aminer/aminer.paper.info.csv')
auth_info = pd.read_csv('/home/duyongkang/aminer/aminer.author.expertise.tag.csv')


In [2]:
#先把全部学者编号和标签拿出来放到list中
ppr_list = info['paper_id'].values.tolist()             # 论文列表
text_list = info['text'].values.tolist()                # 文本列表
auth_list = auth_info['author_id'].values.tolist()      # 作者列表
exper_list = auth_info['expertise_tag'].values.tolist() # 专长列表

In [3]:
auth_info

,author_id,expertise_tag
0,678283,organization quality models
1,678283,software artifacts
2,678283,gsfc software engineering laboratory sel
3,678283,definition cots level design integration
4,678283,functional specifications
...,...,...
233862,62731,deformation energy
233863,62731,intersecting models
233864,62731,tetrahedral mesh
233865,62731,intersecting surfaces


In [5]:
auth_paper = pd.read_csv('/home/duyongkang/aminer/aminer.author.paper.link.csv')
auth_paper

,paper_id,author_id,position
0,782,678283,0
1,872,644976,1
2,872,107969,2
3,977,16959,1
4,1014,500399,1
...,...,...,...
102929,2078596,846386,3
102930,2092355,945190,1
102931,2092355,175920,5
102932,2092355,345733,7


In [4]:
info

,paper_id,text,conference,citations
0,904183,kid's programming language (kpl) in this paper...,acm siggraph educators program,857063
1,1354235,experimental studies on timing and memory usag...,fuzzieee proceedings of international conferen...,401258
2,1426623,movemine: mining moving object databases with ...,proceedings of acm sigmod international confer...,-
3,1912656,combining multi-level evidence for medical rec...,proceedings of international workshop on smart...,976952;1277476
4,737382,efficient aes implementations for arm based pl...,proceedings of acm symposium on applied computing,-
...,...,...,...,...
76428,1431966,bio-signal integration for humanoid operation:...,iconip proceedings of international conference...,1129233
76429,1995475,influence of controlled and uncontrolled inter...,proceedings of ieeeacm international conferenc...,1746850
76430,379274,"a large, fast instruction window for toleratin...",isca proceedings of annual symposium on comput...,1309117;396803;304023
76431,1330531,application-driven statistical paraphrase gene...,acl proceedings of joint conference of annual ...,1271720;816156;854925


In [3]:
#去掉标点符号并分割成单词
lists_new = []
for list in text_list:
    string = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", " ", list)
    lists_new.append(string.split())

## 训练Glove模型

In [4]:
from __future__ import print_function
import argparse
import pprint
import gensim
from glove import Glove
from glove import Corpus

corpus_model = Corpus()
corpus_model.fit(lists_new, window=10)
print('Dict size: %s' % len(corpus_model.dictionary))
print('Collocations: %s' % corpus_model.matrix.nnz)

Dict size: 194506
Collocations: 19985596


In [ ]:
glove = Glove(no_components=100, learning_rate=0.01)
glove.fit(corpus_model.matrix, epochs=2000, no_threads=10, verbose=True)
glove.add_dictionary(corpus_model.dictionary)

Performing 2000 training epochs with 10 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Ep

In [ ]:
glove.save('glove.model')
glove = Glove.load('glove.model')

## 生成论文向量

In [ ]:
# 合并每个学者的专长
exper_dic = {}   #key为学者编号，value为专长
for i in range(len(auth_list)):
    node = "a"+str(auth_list[i])
    if node not in exper_dic:
        exper_dic[node] = []
#     print(exper_list[i])
    temp = exper_list[i].split()
#     print(temp)
    exper_dic[node].extend(temp)

In [ ]:
#专长向量化
exper_vec = {}
for key in exper_dic:        # 每个专家节点           
    exper_vec[key] = []
    num_of_words = len(exper_dic[key])
    for i in range(100):    #100维
        value = 0
        for doc in exper_dic[key]: # 每个词
            try:
                vec = glove.word_vectors[glove.dictionary[doc]][i]
                value+=vec
            except:
                continue
        value /= num_of_words
        exper_vec[key].append(value)
# 生成dataframe文件
df_a = pd.DataFrame(pd.Series(exper_vec), columns=['vector'])
df_a = df_a.reset_index().rename(columns = {'index':'node'})

## 构建论文向量

In [ ]:
#构建一个字典，key为论文编号，value为text
ppr_dic = {}
for i in range(len(ppr_list)):
    node = "p"+str(ppr_list[i])
    if node not in ppr_dic:
        ppr_dic[node] = []
    ppr_dic[node].extend(lists_new[i])

In [ ]:
#生成论文的文本特征
ppr_vec = {}
for key in ppr_dic:
    ppr_vec[key] = []
    num_of_words = len(ppr_dic[key])
    for i in range(100):
        value = 0
        for doc in ppr_dic[key]:
            try:
                vec = glove.word_vectors[glove.dictionary[doc]][i]
            except:
                continue
            value+=vec
        value /= num_of_words
        ppr_vec[key].append(value)

In [16]:
ppr_vec

{'p904183': [0.13129625694701808,
  -0.1437388450769978,
  -0.13545154057773354,
  0.10591299827429239,
  0.1481952726317179,
  0.16119172243005342,
  0.06899791646641305,
  0.10672353178709106,
  0.13994765430118314,
  0.1294430626720729,
  -0.19676938225615445,
  -0.15600582155317536,
  -0.17123828313604916,
  0.09545324213703223,
  0.22129160498066477,
  0.11849628651138294,
  0.14571348443157564,
  -0.2524224868355177,
  0.19665851711926427,
  -0.24538393232147682,
  -0.13473370516978236,
  0.13151809546860949,
  0.19716147518031957,
  -0.07516667804507109,
  -0.118922014372668,
  0.17502859776457233,
  -0.2429642035745565,
  0.14541578555799226,
  -0.19135844619335698,
  -0.2209775338238668,
  0.17151116301272726,
  -0.20216651135113936,
  -0.20479028047739617,
  0.16261231016478137,
  0.17814139044756652,
  0.0291376309570075,
  0.2832890855358837,
  -0.07521741431250355,
  0.16800980921118597,
  -0.057707772676101825,
  -0.13377954680398046,
  -0.029453556255113894,
  0.16089607

In [17]:
# 生成dataframe文件
df_p = pd.DataFrame(pd.Series(ppr_vec), columns=['vector'])
df_p = df_p.reset_index().rename(columns = {'index':'node'})

In [18]:
df_p

,node,vector
0,p904183,"[0.13129625694701808, -0.1437388450769978, -0...."
1,p1354235,"[0.20720332749045522, -0.1628703389539524, -0...."
2,p1426623,"[0.15004017889752375, -0.18323280370634248, -0..."
3,p1912656,"[0.1518085498706247, -0.22623813412560628, -0...."
4,p737382,"[0.19174778147889626, -0.19988718516186565, -0..."
...,...,...
76428,p1431966,"[0.03777086896674449, -0.16204416693009452, -0..."
76429,p1995475,"[0.141152415126573, -0.19769677519527767, -0.1..."
76430,p379274,"[0.18149931838638564, -0.17801616406127477, -0..."
76431,p1330531,"[0.15224532785183575, -0.22790579462891103, -0..."


In [14]:
result

,node,vector
0,a678283,"[0.16741463116595254, -0.08847874727516288, -0..."
1,a644976,"[-0.014381148492826628, -0.10900199397813291, ..."
2,a107969,"[0.11229567620472136, -0.06890523763712762, 0...."
3,a16959,"[0.01676454270364003, -0.28481575581784246, 0...."
4,a500399,"[0.19352898242879324, -0.10215860355813833, -0..."
...,...,...
28581,a828319,"[-0.06098380311722334, -0.10164132889892571, -..."
28582,a1451017,"[-0.06098380311722334, -0.10164132889892571, -..."
28583,a351825,"[0.11646040603768157, -0.05173670195865413, -0..."
28584,a12321,"[0.2256526103294584, -0.08897212559328437, -0...."


In [19]:
# 合并作者向量和论文向量
result = df_a.append(df_p)
result.to_pickle('Glove.pkl')

In [20]:
result

,node,vector
0,a678283,"[0.16741463116595254, -0.08847874727516288, -0..."
1,a644976,"[-0.014381148492826628, -0.10900199397813291, ..."
2,a107969,"[0.11229567620472136, -0.06890523763712762, 0...."
3,a16959,"[0.01676454270364003, -0.28481575581784246, 0...."
4,a500399,"[0.19352898242879324, -0.10215860355813833, -0..."
...,...,...
76428,p1431966,"[0.03777086896674449, -0.16204416693009452, -0..."
76429,p1995475,"[0.141152415126573, -0.19769677519527767, -0.1..."
76430,p379274,"[0.18149931838638564, -0.17801616406127477, -0..."
76431,p1330531,"[0.15224532785183575, -0.22790579462891103, -0..."
